# HW 2

Due 3/24/22

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.utils.data as Data

## 1.1 Data preprocessing

The CSV file should be in the same directory as this notebook. Otherwise, update the file_name path.

References:
* https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [2]:
file_name = './Untitled spreadsheet - Terres_Li_Sample_Data_Set.csv'
df = pd.read_csv(file_name)

In [3]:
# Independent variables
indep_vars = ['Bx', 'By', 'Bz', 'Bmag_avg', 'Ni_avg', 'Vx', 'Vy', 'Vz', 'VSW', 
'Vth_mag_Ion', 'Vth_para_Ion', 'Vth_perp_Ion', 'Ne', 'Ue', 'Te', 'Ti', 'Ti_para', 
'Ti_perp']

# Derived variables: from independent variables using formulas
deriv_vars = ['VA', 'Beta', 'Beta_para', 'Beta_perp', 'Omega_i', 'omega_pi', 
'omega_pe', 'rho_i', 'rho_s', 'd_i', 'd_e', 'sigma_i', 'Lperp', 'lambda_d']

# Dependent variable: target
dep_var = ['k_brk']

print(len(indep_vars), len(deriv_vars), len(dep_var))

18 14 1


In [104]:
# Convert Pandas dataframes to PyTorch tensors
indep = torch.tensor(df[indep_vars].values)
deriv = torch.tensor(df[deriv_vars].values)
dep = torch.tensor(df[dep_var].values)

# Normalize the input data along each variable
indep = torch.nn.functional.normalize(indep, dim=0)
deriv = torch.nn.functional.normalize(deriv, dim=0)

indep_dep = Data.TensorDataset(indep, dep)

In [117]:
dep.mean()

tensor(4.8942e-06, dtype=torch.float64)

In [106]:
# Decide on training, validation, and testing splits
train_size = int(0.70 * len(df))
valid_size = int(0.15 * len(df)) + 1
test_size = len(df) - (train_size + valid_size)
splits = [train_size, valid_size, test_size]

print('Training, validation, testing:', splits)

# Split the data
train_set, valid_set, test_set = Data.random_split(indep_dep, splits)

Training, validation, testing: [4923, 1055, 1055]


In [107]:
# Define dataloaders
batch_size = 64
train_loader = Data.DataLoader(
    dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=2
)
valid_loader = Data.DataLoader(
    dataset=valid_set, batch_size=batch_size, shuffle=True, num_workers=2
)
test_loader = Data.DataLoader(
    dataset=test_set, batch_size=batch_size, shuffle=True, num_workers=2
)

## 1.2 Model definition

References:
* https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html
* https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

In [129]:
class Net1(nn.Module): 
    def __init__(self, num_features, num_hidden, num_output):
        super(Net1, self).__init__()
        self.stack = nn.Sequential(
            nn.Linear(num_features, num_hidden),
            nn.ReLU(),
            nn.Linear(num_hidden, num_output)
        )
    def forward(self, x):
        logits = self.stack(x)
        return logits

class Net2(nn.Module): 
    def __init__(self, num_features, num_hiddens, num_output):
        super(Net2, self).__init__()
        self.stack = nn.Sequential(
            nn.Linear(num_features, num_hiddens[0]),
            nn.ReLU(),
            nn.Linear(num_features, num_hiddens[1]),
            nn.ReLU(),
            nn.Linear(num_hiddens[1], num_output)
        )
    def forward(self, x):
        logits = self.stack(x)
        return logits


In [ ]:
# Define train & test functions for one epoch
def train_loop(loader, model, loss_fun, optimizer):
    train_loss = 0.0
    for X, y in loader:
        prediction = model(X)           # forward pass
        loss = loss_fun(prediction, y)  # compute loss
        train_loss += loss.item()
        optimizer.zero_grad()           # clear gradients for this loop
        loss.backward()                 # compute gradients
        optimizer.step()                # apply gradients
    return train_loss/len(loader)
    

def test_loop(loader, model, loss_fun):
    test_loss = 0.0
    with torch.no_grad():
        for X, y in loader:
            prediction = model(X)
            test_loss += loss_fun(prediction, y).item()
    return test_loss/len(loader)
    

In [149]:
loss_fun = nn.MSELoss()
model_16 = Net1(len(indep_vars), 16, 1)
opt_16 = torch.optim.Adam(model_16.parameters(), lr=0.1)

for epoch in range(5):
    a = train_loop(train_loader, model_16.double(), loss_fun, opt_16)
    b = test_loop(valid_loader, model_16.double(), loss_fun)
    print(a, b)


17
1.6099813263237784e-06 1.8251072668427603e-06
17
1.6138504453689187e-07 1.6409830779303493e-07
17
9.749436123413298e-08 7.593801690739161e-08
17
5.057951907038824e-08 5.9292662759514147e-08
17
5.554590383370939e-08 4.587498616378791e-08
17
4.600273273182674e-08 3.866575873095302e-08
17
4.12938531076328e-08 4.200182821441308e-08
17
2.800408798035257e-08 2.6272254432860266e-08
17
2.9505238014292574e-08 2.247894551236902e-08
17
2.9428859081072812e-08 1.950293470067712e-08
